In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('retail_transactions.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,546729,22775,PURPLE DRAWERKNOB ACRYLIC EDWARDIAN,12,2011-03-16 11:36:00,1.25,18231.0,United Kingdom
1,559898,21868,POTTING SHED TEA MUG,6,2011-07-13 12:18:00,1.25,16225.0,United Kingdom
2,548648,71459,HANGING JAM JAR T-LIGHT HOLDER,24,2011-04-01 13:20:00,0.85,12949.0,United Kingdom
3,540543,22173,METAL 4 HOOK HANGER FRENCH CHATEAU,4,2011-01-09 15:23:00,2.95,14395.0,United Kingdom
4,561390,20726,LUNCH BAG WOODLAND,10,2011-07-27 09:52:00,1.65,17068.0,United Kingdom


In [3]:
df['InvoiceDate'] = pd.to_datetime(df.InvoiceDate)
df['revenue'] = df.UnitPrice * df.Quantity
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
0,546729,22775,PURPLE DRAWERKNOB ACRYLIC EDWARDIAN,12,2011-03-16 11:36:00,1.25,18231.0,United Kingdom,15.0
1,559898,21868,POTTING SHED TEA MUG,6,2011-07-13 12:18:00,1.25,16225.0,United Kingdom,7.5
2,548648,71459,HANGING JAM JAR T-LIGHT HOLDER,24,2011-04-01 13:20:00,0.85,12949.0,United Kingdom,20.4


In [4]:
operations = {'revenue' : 'sum',
            'InvoiceDate' : 'first',
            'CustomerID': 'first'}
df = df.groupby(['InvoiceNo']).agg(operations)
df.head(3)

,revenue,InvoiceDate,CustomerID
InvoiceNo,,,
536365,139.12,2010-12-01 08:26:00,17850.0
536366,22.20,2010-12-01 08:28:00,17850.0
536367,278.73,2010-12-01 08:34:00,13047.0


In [5]:
df['year'] = df.InvoiceDate.apply(lambda x: x.year)
df['days_since'] = (pd.datetime(year = 2010, month = 12, day = 31) - df.InvoiceDate).apply(lambda x: x.days)

/tmp/ipykernel_129070/3296690281.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df['days_since'] = (pd.datetime(year = 2010, month = 12, day = 31) - df.InvoiceDate).apply(lambda x: x.days)


In [6]:
operations = {'revenue' : 'sum',
            'days_since': ['max','min','nunique']}
X = df[df.year == 2010].groupby(['CustomerID']).agg(operations)
X.columns = [' '.join(col).strip() for col in X.columns.values]
X.head(3)

,revenue sum,days_since max,days_since min,days_since nunique
CustomerID,,,,
12347.0,711.79,23,23,1
12348.0,892.80,14,14,1
12370.0,1868.02,16,13,2


In [7]:
X['avg_order_cost'] = X['revenue sum'] / X['days_since nunique']
y = df[df.year == 2011].groupby(['CustomerID'])['revenue'].sum()
wrangled_df = pd.concat([X,y], axis = 1)
wrangled_df.columns = ['2010 revenue', 
                        'days_since_first_purchase', 
                        'days_since_last_purchase', 
                        'number_of_purchases', 
                        'avg_order_cost', 
                        '2011 revenue']
wrangled_df.head(3)

,2010 revenue,days_since_first_purchase,days_since_last_purchase,number_of_purchases,avg_order_cost,2011 revenue
CustomerID,,,,,,
12346.0,NaN,NaN,NaN,NaN,NaN,77183.60
12347.0,711.79,23.0,23.0,1.0,711.79,3598.21
12348.0,892.80,14.0,14.0,1.0,892.80,904.44


In [ ]:
wrangled_df = wrangled_df[~wrangled_df['2010 revenue'].isnull()]
wrangled_df = wrangled_df[~wrangled_df['2011 revenue'].isnull()]

In [8]:
## Eliminar datos atipicos de 2010 revenue y 2011 revenue con mas de 3 desviaciones estandar
wrangled_df = wrangled_df[wrangled_df['2011 revenue'] < ((wrangled_df['2011 revenue'].median()) + wrangled_df['2011 revenue'].std() *3)]
wrangled_df = wrangled_df[wrangled_df['2010 revenue'] < ((wrangled_df['2010 revenue'].median()) + wrangled_df['2010 revenue'].std() *3)]
wrangled_df.to_csv('wrangled_transactions.csv')
wrangled_df.head(3)

,2010 revenue,days_since_first_purchase,days_since_last_purchase,number_of_purchases,avg_order_cost,2011 revenue
CustomerID,,,,,,
12347.0,711.79,23.0,23.0,1.0,711.79,3598.21
12348.0,892.80,14.0,14.0,1.0,892.80,904.44
12370.0,1868.02,16.0,13.0,2.0,934.01,1677.67


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ff5e2091-543c-4b2c-9a56-1e41bbaff775' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>